# Object of this kernel

- to get more people to involved in this competition (less participants to get overwhelmed in the beginning)
- to explain how the submission process works (because I struggled understanding it)

# Install GFootball Environment

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

# Check the observations

In [ ]:
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True}, debug=True)
obs = env.reset()
print(obs[0].keys())
print(len(obs[0]['observation']['players_raw'][0].keys()))

# Create a submission file

In [ ]:
%%writefile submission.py
'''1. This cell only create a new file submission.py and there is no effect on any variables'''
'''We `writefile` because we submit a python file to get evaluated'''

#
# https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/football/helpers.py
from kaggle_environments.envs.football.helpers import Action, human_readable_agent

import numpy as np
import sys

dummy = 0

In [ ]:
print(dummy)

See? The cell above has NO EFFECT on cell outside of it.

# Append code to the submission file

In [ ]:
%%writefile -a submission.py
'''2. and this appends below code to the existing python file (submission.py). Still it does not anything to existing variables'''

# To see the progress
i = 0

# documentation for @human_readable_agent
# https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/football/test_football.py

@human_readable_agent
def agent(obs):
    
    global i

    # Appears in `env.run()`
    sys.stdout.write('\rRendering video... {}/3000'.format(i))
    i += 1
    
    # Make sure player is running.
    if Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint

    # We only can manipulate a single player at the same time
    # ref:https://www.kaggle.com/c/google-football/discussion/188474#1036499
    controlled_player_pos = obs['left_team'][obs['active']]

    #OFFENSE
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        
        # controlled player is close enough to the goal 
        # (would need to calculate actual distance, not difference in x)
        if controlled_player_pos[0] > 0.6:
            return Action.Shot
        
        # Otherwise passes 70%
        if np.random.rand() < 0.7:
            return Action.ShortPass
        
        return Action.Right
    
    #DEFENSE
    else:
        # Run towards the ball.
        if obs['ball'][0] > controlled_player_pos[0] + 0.05:
            return Action.Right
        if obs['ball'][0] < controlled_player_pos[0] - 0.05:
            return Action.Left
        if obs['ball'][1] > controlled_player_pos[1] + 0.05:
            return Action.Bottom
        if obs['ball'][1] < controlled_player_pos[1] - 0.05:
            return Action.Top
        # Try to take over the ball if close to the ball.
        return Action.Slide

In [ ]:
# Set up the Environment again.
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True}, debug=True)

# You could use "do_nothing" for `right_team` or another python file.
left_team = "/kaggle/working/submission.py"
right_team = "/kaggle/working/submission.py"

# Run the simulator
output = env.run([left_team, right_team])[-1]

# Match result. `info` is empty unless it is interrupted by exceptions or timeout.
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))

# Finally,
# watch a match
# on the couch!!
env.render(mode="human", width=800, height=600) 